In [1]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID" # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]="0"

# Importing the libraries 
import torch
import torchvision
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import numpy as np
import matplotlib.pyplot as plt
import torch.nn as nn
import matplotlib.cm as cm
import pandas as pd
import os
import librosa
import librosa.display
import torch.nn.functional as F
from torch.autograd import Variable
import time
from Models import Autoencoder
from TestData import SingleMus
import IPython.display as ipd

# Checking the availability of CUDA on machine
print("Torch", torch.__version__, "CUDA", torch.version.cuda)
print("Device:", torch.device("cuda:0"))
print(torch.cuda.is_available())



/media/sdb1/hy17/venv/lib/python3.6/site-packages/numba/errors.py:131: UserWarning: Insufficiently recent colorama version found. Numba requires colorama >= 0.3.9
  warnings.warn(msg)


Torch 1.2.0 CUDA 10.0.130
Device: cuda:0
True


## Data Preparation

In [7]:
# path = '/media/sdc1/Data/ETRI_Music/'
# data = SingleMus(path)
# path = '/media/sdc1/Data/ETRI_Music'
# bs = 50
# train_loader = torch.utils.data.DataLoader(data, batch_size = bs, shuffle \
#                                            = True, num_workers = 4)
# torch.save(train_loader, 'mus_dataloader.pth')

In [3]:
train_loader = torch.load('mus_train_dataloader.pth')

In [2]:
model = Autoencoder().cuda()
# print(model.code.is_leaf)
lr = 0.0001       
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(list(model.parameters()) + [model.code], lr=lr)
# ,
#                              weight_decay=1e-5)

## Train

In [9]:
epochs = 50
for t in range(epochs):
    start = time.time()
    model.train()
    for data in train_loader:
#         print(data.shape)
        data = data.cuda()
        outputs = model(data)
#         print(outputs[0])
        optimizer.zero_grad()
        loss = criterion(outputs, data)
#         print(loss)
        loss.backward()
#         print(list(model.code.grad))
        optimizer.step()
    
    print('epoch_{}'.format(t))
    end = time.time()
    
    print('loss:' '{:.3f}'.format(loss.data.cpu().numpy()))
    print(test())
#           , '{:.0f}'.format(end-start))
#     if t == 1:
#         break


epoch_0
loss:0.018
2.1927540099636493
epoch_1
loss:0.006
7.929855593023728
epoch_2
loss:0.003
10.716008498784388
epoch_3
loss:0.003
12.025760122164074
epoch_4
loss:0.002
12.83340691608995
epoch_5
loss:0.003
13.422787917379797
epoch_6
loss:0.002
13.857657442227344
epoch_7
loss:0.002
14.28942386089464
epoch_8
loss:0.001
14.579789301294172
epoch_9
loss:0.002
14.672942026749055
epoch_10
loss:0.002
15.003560680730482
epoch_11
loss:0.001
15.160499424154573
epoch_12
loss:0.001
15.331202014765699
epoch_13
loss:0.001
15.376262043946072
epoch_14
loss:0.001
15.538053807583115
epoch_15
loss:0.002
15.657746853325664
epoch_16
loss:0.002
15.851388573810837
epoch_17
loss:0.002
15.9413798037576
epoch_18
loss:0.001
16.14185683461943
epoch_19
loss:0.001
16.317714240164932
epoch_20
loss:0.001
16.470982682262687
epoch_21
loss:0.001
16.588339384263783
epoch_22
loss:0.001
16.834892163573123
epoch_23
loss:0.001
17.037524769450627
epoch_24
loss:0.001
17.216615693946373
epoch_25
loss:0.002
17.337689980583924
ep

In [ ]:
torch.save(model.state_dict(), '0213.model')  

## Test

In [4]:
# path = '/media/sdc1/Data/ETRI_Music/'
# test_data = SingleMus(path)
# bs = 50
# test_loader = torch.utils.data.DataLoader(test_data, batch_size = bs, shuffle \
#                                          = True, num_workers = 4)
# torch.save(test_loader, 'mus_test_dataloader.pth')

In [5]:
test_loader = torch.load('mus_test_dataloader.pth')

In [6]:
# model = Autoencoder().cuda()
# model.load_state_dict(torch.load('0213.model'))

In [7]:
def SDR(s, sr): # input (50, 512), (50, 512)
    sdrs = []
    s = s.cpu().data.numpy()
    sr = sr.cpu().data.numpy()
    for i in range(len(s)):
        eps=1e-20
        sdrs.append(10*np.log10(np.sum(sr**2)/(np.sum((s-sr)**2)+eps)+eps))
    return np.mean(sdrs)

In [8]:
# model.eval()
def test():
    sdr_list = []
    i = 0
    for data in test_loader:
#         print(data)
        data = data.cuda()
        outputs = model.forward(data)
#         print(outputs)
        sdr_list.append(SDR(data, outputs))
    sdr_score = np.mean(sdr_list)
    
    return sdr_score

In [12]:
x = np.array([[1,3]])
y = np.array([[3],[4]])

In [16]:
(x-y)**2

array([[4, 0],
       [9, 1]])

In [17]:
# samples - 3; features - 2(embeddings)
x = torch.Tensor([[1,2], [3,4], [5,6]])

In [33]:
torch.sum(torch.sub(x[None,:,:],x[:, None, :])**2, dim = -1)

tensor([[ 0.,  8., 32.],
        [ 8.,  0.,  8.],
        [32.,  8.,  0.]])

In [32]:
torch.sub(x[None,:,:],x[:, None, :]).shape

torch.Size([3, 3, 2])

In [31]:
x[:, None, :]

tensor([[[1., 2.]],

        [[3., 4.]],

        [[5., 6.]]])